In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pprint
from IPython import display
from pathlib import *
import pandas as pd
import xarray as xr
from bokeh.palettes import all_palettes
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from climatools.plot.plot import *
from climatools.atm import *

from exp.nb_00 import *
from exp.nb_01 import *
from exp.nb_02 import *

output_notebook()

Loading BokehJS ...

In [2]:
! pwd

/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev


In [3]:
PARAM = LBLnewBestfitSWParam(**{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0})

In [4]:
PATH = Path("test1/test1")

In [5]:
np.random.seed(30)

## CRD heating rate for each $g$

In [6]:
heat = pd.read_csv(PATH/'fort.401', header=None, sep=r'\s+')
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns.names = ['g']

pltdata = [{'srs':xr.DataArray(heat[g]),
            'label':f'{g}',
            'color':np.random.choice(all_palettes['Category20'][15]),
            'alpha': np.random.uniform(.8, 1.),
            'line_width':np.random.uniform(2, 7),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for g in heat]

fig_linp = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat', prange=(50, 1050))
fig_logp = plt_vert_profile_bokeh(pltdata=pltdata, y_axis_type='log', xlabel='heat', prange=(1e-2, 200))
show(gridplot(fig_linp, fig_logp, ncols=2))

## WGT heating rate for each $g$

In [7]:
heatg = pd.read_csv(PATH/'fort.400', header=None, sep=r'\s+')
heatg.set_index(0, inplace=True)
heatg.index.names = ['pressure']
heatg.columns.names = ['g']

pltdata = [{'srs':xr.DataArray(heatg[g]),
            'label':f'{g}',
            'color':np.random.choice(all_palettes['Category20'][15]),
            'alpha':np.random.uniform(.8, 1.),
            'line_width':np.random.uniform(2, 7),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for g in heatg]

fig_linp = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heatg', prange=(50, 1050))
fig_logp = plt_vert_profile_bokeh(pltdata=pltdata, y_axis_type='log', xlabel='heatg', prange=(1e-2, 200))
show(gridplot(fig_linp, fig_logp, ncols=2))

## (CRD vs WGT)  heating rate for each $g$

In [8]:
heatg = pd.read_csv(PATH/'fort.400', header=None, sep=r'\s+')
heatg.set_index(0, inplace=True)
heatg.index.names = ['pressure']
heatg.columns.names = ['g']

heat = pd.read_csv(PATH/'fort.401', header=None, sep=r'\s+')
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns.names = ['g']

for g in heat.columns:
    pltdata = [{'srs':xr.DataArray(heatg[g]),
                'label':f'WGT g={g}',
                'color':all_palettes['Set1'][9][0],
                'alpha':.5,
                'line_width':6,
                'line_dash':'dotdash'},
               {'srs':xr.DataArray(heat[g]),
                'label':f'CRD g={g}',
                'color':all_palettes['Set1'][9][1],
                'alpha':1,
                'line_width':3,
                'line_dash':'solid'}]

    fig_liny = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='Heating Rate', y_axis_type='linear', prange=(50, 1050))
    fig_logy = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='Heating Rate', y_axis_type='log', prange=(1e-2, 200))
    show(gridplot(fig_liny, fig_logy, ncols=2))

## Bands-total heating 

In [9]:
heat = pd.read_csv(PATH/'fort.10', sep=r'\s+', header=None)
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns = ['CRD', 'WGT']

pltdata = [{'srs':xr.DataArray(heat[m]),
            'label':f'{m}',
            'color':np.random.choice(all_palettes['Category10'][3]),
            'alpha':np.random.uniform(.5, .8),
            'line_width':np.random.uniform(3, 6),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for m in heat]

fig_lin = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat', prange=(50, 1050))
fig_log = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat', y_axis_type='log', prange=(1e-2, 200))
show(gridplot(fig_lin, fig_log, ncols=2))

## Flux for each g

In [10]:
def load_flux(fpath=None, name=None):
    '''
    Return flux for each g-group, either calculated by line-by-line (fort.403), 
    or by clirad-sw (fort.402), in a xarray.DataArray.
    '''
    df = pd.read_csv(fpath, sep=r'\s+', header=None)
    df.set_index(0, inplace=True)
    df.index.names = ['pressure']
    df.columns.names = ['g']
    da = xr.DataArray(df)
    da.name = name
    return da

In [11]:
dwgt = load_flux(fpath=PATH/'fort.402', name='wgt')
dcrd = load_flux(fpath=PATH/'fort.403', name='crd')
dwgt = dwgt.isel(pressure=[0, -1]).to_pandas()
dcrd = dcrd.isel(pressure=[0, -1]).to_pandas()
pd.concat([dwgt, dcrd, dwgt - dcrd], axis=0,
          keys=['WGT', 'CRD', 'WGT - CRD'])

g                          1         2         3         4         5   \
          pressure                                                      
WGT       0.0003     0.130107  0.477125  1.778085  4.919086  1.401373   
          1001.1125  0.000000  0.000000  0.000000  0.000005  0.000000   
CRD       0.0003     0.130107  0.477125  1.778085  4.919086  1.401373   
          1001.1125  0.000000  0.000000  0.003754  0.354557  0.000002   
WGT - CRD 0.0003     0.000000  0.000000  0.000000  0.000000  0.000000   
          1001.1125  0.000000  0.000000 -0.003754 -0.354552 -0.000002   

g                          6         7          8          9          10  \
          pressure                                                         
WGT       0.0003     4.278213  9.401966  18.766803  29.432102  37.922961   
          1001.1125  0.000107  0.196829   4.276510  15.896270  29.852561   
CRD       0.0003     4.278213  9.401966  18.766803  29.432102  37.922961   
          1001.1125  0.010440  0.481747   4.682655  15.708362  29.507696   
WGT - CRD 0.0003     0.000000  0.000000   0.000000   0.000000   0.000000   
          1001.1125 -0.010333 -0.284918  -0.406145   0.187907   0.344865   

g                           11          12  
          pressure                          
WGT       0.0003     45.130723  288.781206  
          1001.1125  41.031600  286.019366  
CRD       0.0003     45.130723  288.781206  
          1001.1125  40.900530  285.907942  
WGT - CRD 0.0003      0.000000    0.000000  
          1001.1125   0.131070    0.111424

## Bands-total flux

In [12]:
flux = pd.read_csv(PATH/'fort.9', sep=r'\s+', skiprows=2, header=None)
flux.set_index(0, inplace=True)
flux.index.names = ['pressure']
flux.columns = ['CRD', 'WGT']

pltdata = [{'srs':xr.DataArray(flux[m]),
            'label':f'{m}',
            'color':np.random.choice(all_palettes['Category10'][10]),
            'alpha':np.random.uniform(.5, .8),
            'line_width':np.random.uniform(3, 6),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for m in flux]

fig_lin = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='flux', prange=(50, 1050))
fig_log = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='flux', prange=(1e-2, 200), y_axis_type='log')
show(gridplot(fig_lin, fig_log, ncols=2))

# fin

In [13]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')